In [17]:
from pyspark.sql import SparkSession
# spark = SparkSession.builder.appName('json_parse').getOrCreate()

spark = SparkSession.builder.appName("HighMemorySparkSession").master("yarn").config("spark.executor.memory", "4g").config("spark.executor.cores", "4").config("spark.driver.memory", "8g").config("spark.driver.maxResultSize", "2g").config("spark.executor.instances", "5").config("spark.sql.shuffle.partitions", "200").config("spark.default.parallelism", "200").config("spark.network.timeout", "600s").getOrCreate()

spark.version

25/03/15 17:02:14 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


'3.4.4'

In [7]:
from os import path
from json import dump

weather_data = {'location': {'name': 'Piscataway', 'region': 'New Jersey', 'country': 'USA', 'lat': 40.5514984130859, 'lon': -74.4589996337891, 'tz_id': 'America/New_York', 'localtime_epoch': 1742069043, 'localtime': '2025-03-15 16:04'}, 'current': {'last_updated_epoch': 1742068800, 'last_updated': '2025-03-15 16:00', 'temp_c': 10.3, 'temp_f': 50.5, 'is_day': 1, 'condition': {'text': 'Overcast', 'icon': '//cdn.weatherapi.com/weather/64x64/day/122.png', 'code': 1009}, 'wind_mph': 6.9, 'wind_kph': 11.2, 'wind_degree': 162, 'wind_dir': 'SSE', 'pressure_mb': 1019.0, 'pressure_in': 30.1, 'precip_mm': 0.0, 'precip_in': 0.0, 'humidity': 82, 'cloud': 100, 'feelslike_c': 8.8, 'feelslike_f': 47.8, 'windchill_c': 6.4, 'windchill_f': 43.5, 'heatindex_c': 8.0, 'heatindex_f': 46.5, 'dewpoint_c': 6.4, 'dewpoint_f': 43.5, 'vis_km': 16.0, 'vis_miles': 9.0, 'uv': 1.9, 'gust_mph': 9.2, 'gust_kph': 14.8}, 'forecast': {'forecastday': [{'date': '2025-03-15', 'date_epoch': 1741996800, 'day': {'maxtemp_c': 8.3, 'maxtemp_f': 46.9, 'mintemp_c': 4.7, 'mintemp_f': 40.5, 'avgtemp_c': 6.3, 'avgtemp_f': 43.4, 'maxwind_mph': 9.4, 'maxwind_kph': 15.1, 'totalprecip_mm': 0.0, 'totalprecip_in': 0.0, 'totalsnow_cm': 0.0, 'avgvis_km': 10.0, 'avgvis_miles': 6.0, 'avghumidity': 93, 'daily_will_it_rain': 0, 'daily_chance_of_rain': 0, 'daily_will_it_snow': 0, 'daily_chance_of_snow': 0, 'condition': {'text': 'Overcast ', 'icon': '//cdn.weatherapi.com/weather/64x64/day/122.png', 'code': 1009}, 'uv': 1.0}, 'astro': {'sunrise': '07:09 AM', 'sunset': '07:05 PM', 'moonrise': '08:40 PM', 'moonset': '07:38 AM', 'moon_phase': 'Waning Gibbous', 'moon_illumination': 100, 'is_moon_up': 1, 'is_sun_up': 0}, 'hour': [{'time_epoch': 1742011200, 'time': '2025-03-15 00:00', 'temp_c': 5.7, 'temp_f': 42.2, 'is_day': 0, 'condition': {'text': 'Mist', 'icon': '//cdn.weatherapi.com/weather/64x64/night/143.png', 'code': 1030}, 'wind_mph': 5.4, 'wind_kph': 8.6, 'wind_degree': 170, 'wind_dir': 'S', 'pressure_mb': 1021.0, 'pressure_in': 30.14, 'precip_mm': 0.0, 'precip_in': 0.0, 'snow_cm': 0.0, 'humidity': 94, 'cloud': 100, 'feelslike_c': 3.8, 'feelslike_f': 38.8, 'windchill_c': 3.8, 'windchill_f': 38.8, 'heatindex_c': 5.7, 'heatindex_f': 42.2, 'dewpoint_c': 4.9, 'dewpoint_f': 40.7, 'will_it_rain': 0, 'chance_of_rain': 0, 'will_it_snow': 0, 'chance_of_snow': 0, 'vis_km': 10.0, 'vis_miles': 6.0, 'gust_mph': 7.7, 'gust_kph': 12.3, 'uv': 0}, {'time_epoch': 1742014800, 'time': '2025-03-15 01:00', 'temp_c': 5.5, 'temp_f': 41.9, 'is_day': 0, 'condition': {'text': 'Mist', 'icon': '//cdn.weatherapi.com/weather/64x64/night/143.png', 'code': 1030}, 'wind_mph': 4.7, 'wind_kph': 7.6, 'wind_degree': 162, 'wind_dir': 'SSE', 'pressure_mb': 1020.0, 'pressure_in': 30.13, 'precip_mm': 0.0, 'precip_in': 0.0, 'snow_cm': 0.0, 'humidity': 95, 'cloud': 100, 'feelslike_c': 3.8, 'feelslike_f': 38.8, 'windchill_c': 3.8, 'windchill_f': 38.8, 'heatindex_c': 5.5, 'heatindex_f': 41.9, 'dewpoint_c': 4.7, 'dewpoint_f': 40.5, 'will_it_rain': 0, 'chance_of_rain': 0, 'will_it_snow': 0, 'chance_of_snow': 0, 'vis_km': 10.0, 'vis_miles': 6.0, 'gust_mph': 6.7, 'gust_kph': 10.8, 'uv': 0}, {'time_epoch': 1742018400, 'time': '2025-03-15 02:00', 'temp_c': 5.3, 'temp_f': 41.6, 'is_day': 0, 'condition': {'text': 'Mist', 'icon': '//cdn.weatherapi.com/weather/64x64/night/143.png', 'code': 1030}, 'wind_mph': 4.5, 'wind_kph': 7.2, 'wind_degree': 179, 'wind_dir': 'S', 'pressure_mb': 1020.0, 'pressure_in': 30.13, 'precip_mm': 0.0, 'precip_in': 0.0, 'snow_cm': 0.0, 'humidity': 96, 'cloud': 100, 'feelslike_c': 3.6, 'feelslike_f': 38.5, 'windchill_c': 3.6, 'windchill_f': 38.5, 'heatindex_c': 5.3, 'heatindex_f': 41.6, 'dewpoint_c': 4.7, 'dewpoint_f': 40.4, 'will_it_rain': 0, 'chance_of_rain': 0, 'will_it_snow': 0, 'chance_of_snow': 0, 'vis_km': 10.0, 'vis_miles': 6.0, 'gust_mph': 6.5, 'gust_kph': 10.4, 'uv': 0}, {'time_epoch': 1742022000, 'time': '2025-03-15 03:00', 'temp_c': 5.2, 'temp_f': 41.4, 'is_day': 0, 'condition': {'text': 'Mist', 'icon': '//cdn.weatherapi.com/weather/64x64/night/143.png', 'code': 1030}, 'wind_mph': 4.0, 'wind_kph': 6.5, 'wind_degree': 170, 'wind_dir': 'S', 'pressure_mb': 1020.0, 'pressure_in': 30.12, 'precip_mm': 0.0, 'precip_in': 0.0, 'snow_cm': 0.0, 'humidity': 95, 'cloud': 100, 'feelslike_c': 3.4, 'feelslike_f': 38.1, 'windchill_c': 3.4, 'windchill_f': 38.1, 'heatindex_c': 5.2, 'heatindex_f': 41.4, 'dewpoint_c': 4.5, 'dewpoint_f': 40.1, 'will_it_rain': 0, 'chance_of_rain': 0, 'will_it_snow': 0, 'chance_of_snow': 0, 'vis_km': 10.0, 'vis_miles': 6.0, 'gust_mph': 5.9, 'gust_kph': 9.5, 'uv': 0}, {'time_epoch': 1742025600, 'time': '2025-03-15 04:00', 'temp_c': 5.3, 'temp_f': 41.5, 'is_day': 0, 'condition': {'text': 'Mist', 'icon': '//cdn.weatherapi.com/weather/64x64/night/143.png', 'code': 1030}, 'wind_mph': 3.8, 'wind_kph': 6.1, 'wind_degree': 173, 'wind_dir': 'S', 'pressure_mb': 1020.0, 'pressure_in': 30.11, 'precip_mm': 0.0, 'precip_in': 0.0, 'snow_cm': 0.0, 'humidity': 96, 'cloud': 100, 'feelslike_c': 3.3, 'feelslike_f': 37.9, 'windchill_c': 3.3, 'windchill_f': 37.9, 'heatindex_c': 5.3, 'heatindex_f': 41.5, 'dewpoint_c': 4.5, 'dewpoint_f': 40.1, 'will_it_rain': 0, 'chance_of_rain': 0, 'will_it_snow': 0, 'chance_of_snow': 0, 'vis_km': 10.0, 'vis_miles': 6.0, 'gust_mph': 5.6, 'gust_kph': 9.0, 'uv': 0}, {'time_epoch': 1742029200, 'time': '2025-03-15 05:00', 'temp_c': 5.1, 'temp_f': 41.2, 'is_day': 0, 'condition': {'text': 'Mist', 'icon': '//cdn.weatherapi.com/weather/64x64/night/143.png', 'code': 1030}, 'wind_mph': 3.8, 'wind_kph': 6.1, 'wind_degree': 190, 'wind_dir': 'S', 'pressure_mb': 1020.0, 'pressure_in': 30.11, 'precip_mm': 0.0, 'precip_in': 0.0, 'snow_cm': 0.0, 'humidity': 95, 'cloud': 100, 'feelslike_c': 3.1, 'feelslike_f': 37.6, 'windchill_c': 3.1, 'windchill_f': 37.6, 'heatindex_c': 5.1, 'heatindex_f': 41.2, 'dewpoint_c': 4.6, 'dewpoint_f': 40.3, 'will_it_rain': 0, 'chance_of_rain': 0, 'will_it_snow': 0, 'chance_of_snow': 0, 'vis_km': 10.0, 'vis_miles': 6.0, 'gust_mph': 5.3, 'gust_kph': 8.5, 'uv': 0}, {'time_epoch': 1742032800, 'time': '2025-03-15 06:00', 'temp_c': 5.0, 'temp_f': 40.9, 'is_day': 0, 'condition': {'text': 'Mist', 'icon': '//cdn.weatherapi.com/weather/64x64/night/143.png', 'code': 1030}, 'wind_mph': 3.8, 'wind_kph': 6.1, 'wind_degree': 162, 'wind_dir': 'SSE', 'pressure_mb': 1020.0, 'pressure_in': 30.12, 'precip_mm': 0.0, 'precip_in': 0.0, 'snow_cm': 0.0, 'humidity': 96, 'cloud': 100, 'feelslike_c': 2.9, 'feelslike_f': 37.2, 'windchill_c': 2.9, 'windchill_f': 37.2, 'heatindex_c': 5.0, 'heatindex_f': 40.9, 'dewpoint_c': 4.4, 'dewpoint_f': 40.0, 'will_it_rain': 0, 'chance_of_rain': 0, 'will_it_snow': 0, 'chance_of_snow': 0, 'vis_km': 10.0, 'vis_miles': 6.0, 'gust_mph': 5.3, 'gust_kph': 8.5, 'uv': 0}, {'time_epoch': 1742036400, 'time': '2025-03-15 07:00', 'temp_c': 4.8, 'temp_f': 40.7, 'is_day': 0, 'condition': {'text': 'Mist', 'icon': '//cdn.weatherapi.com/weather/64x64/night/143.png', 'code': 1030}, 'wind_mph': 3.6, 'wind_kph': 5.8, 'wind_degree': 156, 'wind_dir': 'SSE', 'pressure_mb': 1021.0, 'pressure_in': 30.14, 'precip_mm': 0.0, 'precip_in': 0.0, 'snow_cm': 0.0, 'humidity': 96, 'cloud': 100, 'feelslike_c': 2.9, 'feelslike_f': 37.2, 'windchill_c': 2.9, 'windchill_f': 37.2, 'heatindex_c': 4.8, 'heatindex_f': 40.7, 'dewpoint_c': 4.2, 'dewpoint_f': 39.6, 'will_it_rain': 0, 'chance_of_rain': 0, 'will_it_snow': 0, 'chance_of_snow': 0, 'vis_km': 10.0, 'vis_miles': 6.0, 'gust_mph': 5.1, 'gust_kph': 8.2, 'uv': 0}, {'time_epoch': 1742040000, 'time': '2025-03-15 08:00', 'temp_c': 4.9, 'temp_f': 40.8, 'is_day': 1, 'condition': {'text': 'Mist', 'icon': '//cdn.weatherapi.com/weather/64x64/day/143.png', 'code': 1030}, 'wind_mph': 3.8, 'wind_kph': 6.1, 'wind_degree': 146, 'wind_dir': 'SSE', 'pressure_mb': 1021.0, 'pressure_in': 30.15, 'precip_mm': 0.0, 'precip_in': 0.0, 'snow_cm': 0.0, 'humidity': 96, 'cloud': 100, 'feelslike_c': 3.1, 'feelslike_f': 37.5, 'windchill_c': 3.1, 'windchill_f': 37.5, 'heatindex_c': 4.9, 'heatindex_f': 40.8, 'dewpoint_c': 4.1, 'dewpoint_f': 39.3, 'will_it_rain': 0, 'chance_of_rain': 0, 'will_it_snow': 0, 'chance_of_snow': 0, 'vis_km': 10.0, 'vis_miles': 6.0, 'gust_mph': 5.3, 'gust_kph': 8.6, 'uv': 0.2}, {'time_epoch': 1742043600, 'time': '2025-03-15 09:00', 'temp_c': 5.0, 'temp_f': 41.1, 'is_day': 1, 'condition': {'text': 'Mist', 'icon': '//cdn.weatherapi.com/weather/64x64/day/143.png', 'code': 1030}, 'wind_mph': 5.1, 'wind_kph': 8.3, 'wind_degree': 154, 'wind_dir': 'SSE', 'pressure_mb': 1022.0, 'pressure_in': 30.17, 'precip_mm': 0.0, 'precip_in': 0.0, 'snow_cm': 0.0, 'humidity': 95, 'cloud': 100, 'feelslike_c': 3.3, 'feelslike_f': 37.9, 'windchill_c': 3.3, 'windchill_f': 37.9, 'heatindex_c': 5.0, 'heatindex_f': 41.1, 'dewpoint_c': 4.2, 'dewpoint_f': 39.5, 'will_it_rain': 0, 'chance_of_rain': 0, 'will_it_snow': 0, 'chance_of_snow': 0, 'vis_km': 10.0, 'vis_miles': 6.0, 'gust_mph': 7.0, 'gust_kph': 11.3, 'uv': 0.7}, {'time_epoch': 1742047200, 'time': '2025-03-15 10:00', 'temp_c': 5.5, 'temp_f': 41.8, 'is_day': 1, 'condition': {'text': 'Overcast ', 'icon': '//cdn.weatherapi.com/weather/64x64/day/122.png', 'code': 1009}, 'wind_mph': 5.8, 'wind_kph': 9.4, 'wind_degree': 166, 'wind_dir': 'SSE', 'pressure_mb': 1022.0, 'pressure_in': 30.17, 'precip_mm': 0.0, 'precip_in': 0.0, 'snow_cm': 0.0, 'humidity': 93, 'cloud': 100, 'feelslike_c': 3.8, 'feelslike_f': 38.9, 'windchill_c': 3.8, 'windchill_f': 38.9, 'heatindex_c': 5.5, 'heatindex_f': 41.8, 'dewpoint_c': 4.2, 'dewpoint_f': 39.6, 'will_it_rain': 0, 'chance_of_rain': 0, 'will_it_snow': 0, 'chance_of_snow': 0, 'vis_km': 10.0, 'vis_miles': 6.0, 'gust_mph': 7.6, 'gust_kph': 12.2, 'uv': 1.3}, {'time_epoch': 1742050800, 'time': '2025-03-15 11:00', 'temp_c': 5.9, 'temp_f': 42.7, 'is_day': 1, 'condition': {'text': 'Overcast ', 'icon': '//cdn.weatherapi.com/weather/64x64/day/122.png', 'code': 1009}, 'wind_mph': 6.7, 'wind_kph': 10.8, 'wind_degree': 161, 'wind_dir': 'SSE', 'pressure_mb': 1021.0, 'pressure_in': 30.16, 'precip_mm': 0.0, 'precip_in': 0.0, 'snow_cm': 0.0, 'humidity': 90, 'cloud': 100, 'feelslike_c': 4.2, 'feelslike_f': 39.6, 'windchill_c': 4.2, 'windchill_f': 39.6, 'heatindex_c': 5.9, 'heatindex_f': 42.7, 'dewpoint_c': 4.5, 'dewpoint_f': 40.0, 'will_it_rain': 0, 'chance_of_rain': 0, 'will_it_snow': 0, 'chance_of_snow': 0, 'vis_km': 10.0, 'vis_miles': 6.0, 'gust_mph': 8.6, 'gust_kph': 13.9, 'uv': 2.3}, {'time_epoch': 1742054400, 'time': '2025-03-15 12:00', 'temp_c': 6.5, 'temp_f': 43.6, 'is_day': 1, 'condition': {'text': 'Overcast ', 'icon': '//cdn.weatherapi.com/weather/64x64/day/122.png', 'code': 1009}, 'wind_mph': 7.2, 'wind_kph': 11.5, 'wind_degree': 190, 'wind_dir': 'S', 'pressure_mb': 1020.0, 'pressure_in': 30.13, 'precip_mm': 0.0, 'precip_in': 0.0, 'snow_cm': 0.0, 'humidity': 89, 'cloud': 100, 'feelslike_c': 4.8, 'feelslike_f': 40.6, 'windchill_c': 4.8, 'windchill_f': 40.6, 'heatindex_c': 6.5, 'heatindex_f': 43.6, 'dewpoint_c': 4.8, 'dewpoint_f': 40.6, 'will_it_rain': 0, 'chance_of_rain': 0, 'will_it_snow': 0, 'chance_of_snow': 0, 'vis_km': 10.0, 'vis_miles': 6.0, 'gust_mph': 8.8, 'gust_kph': 14.2, 'uv': 3.7}, {'time_epoch': 1742058000, 'time': '2025-03-15 13:00', 'temp_c': 7.1, 'temp_f': 44.7, 'is_day': 1, 'condition': {'text': 'Overcast ', 'icon': '//cdn.weatherapi.com/weather/64x64/day/122.png', 'code': 1009}, 'wind_mph': 6.0, 'wind_kph': 9.7, 'wind_degree': 197, 'wind_dir': 'SSW', 'pressure_mb': 1020.0, 'pressure_in': 30.12, 'precip_mm': 0.0, 'precip_in': 0.0, 'snow_cm': 0.0, 'humidity': 89, 'cloud': 100, 'feelslike_c': 6.1, 'feelslike_f': 43.0, 'windchill_c': 6.1, 'windchill_f': 43.0, 'heatindex_c': 7.1, 'heatindex_f': 44.7, 'dewpoint_c': 5.4, 'dewpoint_f': 41.6, 'will_it_rain': 0, 'chance_of_rain': 0, 'will_it_snow': 0, 'chance_of_snow': 0, 'vis_km': 10.0, 'vis_miles': 6.0, 'gust_mph': 7.6, 'gust_kph': 12.2, 'uv': 4.7}, {'time_epoch': 1742061600, 'time': '2025-03-15 14:00', 'temp_c': 7.6, 'temp_f': 45.8, 'is_day': 1, 'condition': {'text': 'Overcast ', 'icon': '//cdn.weatherapi.com/weather/64x64/day/122.png', 'code': 1009}, 'wind_mph': 5.6, 'wind_kph': 9.0, 'wind_degree': 185, 'wind_dir': 'S', 'pressure_mb': 1021.0, 'pressure_in': 30.15, 'precip_mm': 0.0, 'precip_in': 0.0, 'snow_cm': 0.0, 'humidity': 89, 'cloud': 100, 'feelslike_c': 6.8, 'feelslike_f': 44.2, 'windchill_c': 6.8, 'windchill_f': 44.2, 'heatindex_c': 7.6, 'heatindex_f': 45.8, 'dewpoint_c': 6.0, 'dewpoint_f': 42.7, 'will_it_rain': 0, 'chance_of_rain': 0, 'will_it_snow': 0, 'chance_of_snow': 0, 'vis_km': 10.0, 'vis_miles': 6.0, 'gust_mph': 6.8, 'gust_kph': 10.9, 'uv': 4.3}, {'time_epoch': 1742065200, 'time': '2025-03-15 15:00', 'temp_c': 8.0, 'temp_f': 46.3, 'is_day': 1, 'condition': {'text': 'Overcast ', 'icon': '//cdn.weatherapi.com/weather/64x64/day/122.png', 'code': 1009}, 'wind_mph': 5.6, 'wind_kph': 9.0, 'wind_degree': 175, 'wind_dir': 'S', 'pressure_mb': 1021.0, 'pressure_in': 30.14, 'precip_mm': 0.0, 'precip_in': 0.0, 'snow_cm': 0.0, 'humidity': 87, 'cloud': 100, 'feelslike_c': 6.9, 'feelslike_f': 44.4, 'windchill_c': 6.9, 'windchill_f': 44.4, 'heatindex_c': 8.0, 'heatindex_f': 46.3, 'dewpoint_c': 6.2, 'dewpoint_f': 43.2, 'will_it_rain': 0, 'chance_of_rain': 0, 'will_it_snow': 0, 'chance_of_snow': 0, 'vis_km': 10.0, 'vis_miles': 6.0, 'gust_mph': 7.0, 'gust_kph': 11.2, 'uv': 3.2}, {'time_epoch': 1742068800, 'time': '2025-03-15 16:00', 'temp_c': 10.3, 'temp_f': 50.5, 'is_day': 1, 'condition': {'text': 'Overcast', 'icon': '//cdn.weatherapi.com/weather/64x64/day/122.png', 'code': 1009}, 'wind_mph': 6.9, 'wind_kph': 11.2, 'wind_degree': 162, 'wind_dir': 'SSE', 'pressure_mb': 1019.0, 'pressure_in': 30.1, 'precip_mm': 0.0, 'precip_in': 0.0, 'snow_cm': 0.0, 'humidity': 82, 'cloud': 100, 'feelslike_c': 6.4, 'feelslike_f': 43.5, 'windchill_c': 6.4, 'windchill_f': 43.5, 'heatindex_c': 8.0, 'heatindex_f': 46.5, 'dewpoint_c': 6.4, 'dewpoint_f': 43.5, 'will_it_rain': 0, 'chance_of_rain': 0, 'will_it_snow': 0, 'chance_of_snow': 0, 'vis_km': 16.0, 'vis_miles': 9.0, 'gust_mph': 9.2, 'gust_kph': 14.8, 'uv': 1.9}, {'time_epoch': 1742072400, 'time': '2025-03-15 17:00', 'temp_c': 7.9, 'temp_f': 46.3, 'is_day': 1, 'condition': {'text': 'Overcast ', 'icon': '//cdn.weatherapi.com/weather/64x64/day/122.png', 'code': 1009}, 'wind_mph': 7.8, 'wind_kph': 12.6, 'wind_degree': 150, 'wind_dir': 'SSE', 'pressure_mb': 1019.0, 'pressure_in': 30.1, 'precip_mm': 0.0, 'precip_in': 0.0, 'snow_cm': 0.0, 'humidity': 89, 'cloud': 100, 'feelslike_c': 6.0, 'feelslike_f': 42.8, 'windchill_c': 6.0, 'windchill_f': 42.8, 'heatindex_c': 7.9, 'heatindex_f': 46.3, 'dewpoint_c': 6.3, 'dewpoint_f': 43.4, 'will_it_rain': 0, 'chance_of_rain': 0, 'will_it_snow': 0, 'chance_of_snow': 0, 'vis_km': 10.0, 'vis_miles': 6.0, 'gust_mph': 11.3, 'gust_kph': 18.3, 'uv': 0.8}, {'time_epoch': 1742076000, 'time': '2025-03-15 18:00', 'temp_c': 7.6, 'temp_f': 45.7, 'is_day': 1, 'condition': {'text': 'Overcast ', 'icon': '//cdn.weatherapi.com/weather/64x64/day/122.png', 'code': 1009}, 'wind_mph': 9.4, 'wind_kph': 15.1, 'wind_degree': 143, 'wind_dir': 'SE', 'pressure_mb': 1019.0, 'pressure_in': 30.1, 'precip_mm': 0.0, 'precip_in': 0.0, 'snow_cm': 0.0, 'humidity': 91, 'cloud': 100, 'feelslike_c': 5.4, 'feelslike_f': 41.8, 'windchill_c': 5.4, 'windchill_f': 41.8, 'heatindex_c': 7.6, 'heatindex_f': 45.7, 'dewpoint_c': 6.4, 'dewpoint_f': 43.6, 'will_it_rain': 0, 'chance_of_rain': 0, 'will_it_snow': 0, 'chance_of_snow': 0, 'vis_km': 10.0, 'vis_miles': 6.0, 'gust_mph': 14.0, 'gust_kph': 22.6, 'uv': 0.2}, {'time_epoch': 1742079600, 'time': '2025-03-15 19:00', 'temp_c': 7.3, 'temp_f': 45.1, 'is_day': 1, 'condition': {'text': 'Overcast ', 'icon': '//cdn.weatherapi.com/weather/64x64/day/122.png', 'code': 1009}, 'wind_mph': 9.4, 'wind_kph': 15.1, 'wind_degree': 145, 'wind_dir': 'SE', 'pressure_mb': 1019.0, 'pressure_in': 30.09, 'precip_mm': 0.0, 'precip_in': 0.0, 'snow_cm': 0.0, 'humidity': 94, 'cloud': 100, 'feelslike_c': 5.0, 'feelslike_f': 41.1, 'windchill_c': 5.0, 'windchill_f': 41.1, 'heatindex_c': 7.3, 'heatindex_f': 45.1, 'dewpoint_c': 6.4, 'dewpoint_f': 43.5, 'will_it_rain': 0, 'chance_of_rain': 0, 'will_it_snow': 0, 'chance_of_snow': 0, 'vis_km': 10.0, 'vis_miles': 6.0, 'gust_mph': 14.6, 'gust_kph': 23.5, 'uv': 0.0}, {'time_epoch': 1742083200, 'time': '2025-03-15 20:00', 'temp_c': 7.2, 'temp_f': 45.0, 'is_day': 0, 'condition': {'text': 'Mist', 'icon': '//cdn.weatherapi.com/weather/64x64/night/143.png', 'code': 1030}, 'wind_mph': 8.3, 'wind_kph': 13.3, 'wind_degree': 158, 'wind_dir': 'SSE', 'pressure_mb': 1019.0, 'pressure_in': 30.09, 'precip_mm': 0.0, 'precip_in': 0.0, 'snow_cm': 0.0, 'humidity': 96, 'cloud': 100, 'feelslike_c': 5.1, 'feelslike_f': 41.1, 'windchill_c': 5.1, 'windchill_f': 41.1, 'heatindex_c': 7.2, 'heatindex_f': 45.0, 'dewpoint_c': 6.4, 'dewpoint_f': 43.6, 'will_it_rain': 0, 'chance_of_rain': 0, 'will_it_snow': 0, 'chance_of_snow': 0, 'vis_km': 10.0, 'vis_miles': 6.0, 'gust_mph': 12.7, 'gust_kph': 20.4, 'uv': 0}, {'time_epoch': 1742086800, 'time': '2025-03-15 21:00', 'temp_c': 7.3, 'temp_f': 45.1, 'is_day': 0, 'condition': {'text': 'Mist', 'icon': '//cdn.weatherapi.com/weather/64x64/night/143.png', 'code': 1030}, 'wind_mph': 8.5, 'wind_kph': 13.7, 'wind_degree': 162, 'wind_dir': 'SSE', 'pressure_mb': 1019.0, 'pressure_in': 30.08, 'precip_mm': 0.0, 'precip_in': 0.0, 'snow_cm': 0.0, 'humidity': 97, 'cloud': 100, 'feelslike_c': 5.5, 'feelslike_f': 41.9, 'windchill_c': 5.5, 'windchill_f': 41.9, 'heatindex_c': 7.3, 'heatindex_f': 45.1, 'dewpoint_c': 6.6, 'dewpoint_f': 43.9, 'will_it_rain': 0, 'chance_of_rain': 0, 'will_it_snow': 0, 'chance_of_snow': 0, 'vis_km': 10.0, 'vis_miles': 6.0, 'gust_mph': 13.2, 'gust_kph': 21.2, 'uv': 0}, {'time_epoch': 1742090400, 'time': '2025-03-15 22:00', 'temp_c': 7.4, 'temp_f': 45.3, 'is_day': 0, 'condition': {'text': 'Fog', 'icon': '//cdn.weatherapi.com/weather/64x64/night/248.png', 'code': 1135}, 'wind_mph': 8.3, 'wind_kph': 13.3, 'wind_degree': 164, 'wind_dir': 'SSE', 'pressure_mb': 1019.0, 'pressure_in': 30.09, 'precip_mm': 0.0, 'precip_in': 0.0, 'snow_cm': 0.0, 'humidity': 97, 'cloud': 100, 'feelslike_c': 5.9, 'feelslike_f': 42.6, 'windchill_c': 5.9, 'windchill_f': 42.6, 'heatindex_c': 7.4, 'heatindex_f': 45.3, 'dewpoint_c': 6.9, 'dewpoint_f': 44.4, 'will_it_rain': 0, 'chance_of_rain': 0, 'will_it_snow': 0, 'chance_of_snow': 0, 'vis_km': 10.0, 'vis_miles': 6.0, 'gust_mph': 13.1, 'gust_kph': 21.1, 'uv': 0}, {'time_epoch': 1742094000, 'time': '2025-03-15 23:00', 'temp_c': 7.4, 'temp_f': 45.3, 'is_day': 0, 'condition': {'text': 'Fog', 'icon': '//cdn.weatherapi.com/weather/64x64/night/248.png', 'code': 1135}, 'wind_mph': 7.2, 'wind_kph': 11.5, 'wind_degree': 168, 'wind_dir': 'SSE', 'pressure_mb': 1018.0, 'pressure_in': 30.07, 'precip_mm': 0.0, 'precip_in': 0.0, 'snow_cm': 0.0, 'humidity': 98, 'cloud': 100, 'feelslike_c': 6.1, 'feelslike_f': 43.0, 'windchill_c': 6.1, 'windchill_f': 43.0, 'heatindex_c': 7.4, 'heatindex_f': 45.3, 'dewpoint_c': 7.2, 'dewpoint_f': 44.9, 'will_it_rain': 0, 'chance_of_rain': 0, 'will_it_snow': 0, 'chance_of_snow': 0, 'vis_km': 10.0, 'vis_miles': 6.0, 'gust_mph': 11.8, 'gust_kph': 19.0, 'uv': 0}]}]}}

print(type(weather_data), weather_data)

cwd = "/home/tanvi/Documents/GitRepo/automatic-system/etl/"

with open(path.join(cwd, "weather_data.json"), mode="w") as json_fobj:
    dump(weather_data, json_fobj)


<class 'dict'> {'location': {'name': 'Piscataway', 'region': 'New Jersey', 'country': 'USA', 'lat': 40.5514984130859, 'lon': -74.4589996337891, 'tz_id': 'America/New_York', 'localtime_epoch': 1742069043, 'localtime': '2025-03-15 16:04'}, 'current': {'last_updated_epoch': 1742068800, 'last_updated': '2025-03-15 16:00', 'temp_c': 10.3, 'temp_f': 50.5, 'is_day': 1, 'condition': {'text': 'Overcast', 'icon': '//cdn.weatherapi.com/weather/64x64/day/122.png', 'code': 1009}, 'wind_mph': 6.9, 'wind_kph': 11.2, 'wind_degree': 162, 'wind_dir': 'SSE', 'pressure_mb': 1019.0, 'pressure_in': 30.1, 'precip_mm': 0.0, 'precip_in': 0.0, 'humidity': 82, 'cloud': 100, 'feelslike_c': 8.8, 'feelslike_f': 47.8, 'windchill_c': 6.4, 'windchill_f': 43.5, 'heatindex_c': 8.0, 'heatindex_f': 46.5, 'dewpoint_c': 6.4, 'dewpoint_f': 43.5, 'vis_km': 16.0, 'vis_miles': 9.0, 'uv': 1.9, 'gust_mph': 9.2, 'gust_kph': 14.8}, 'forecast': {'forecastday': [{'date': '2025-03-15', 'date_epoch': 1741996800, 'day': {'maxtemp_c': 8.

In [10]:
jdf = spark.read.json(path.join(cwd, "weather_data.json"))

jdf.show()

+--------------------+--------------------+--------------------+
|             current|            forecast|            location|
+--------------------+--------------------+--------------------+
|{100, {1009, //cd...|{[{{1, 0, 100, Wa...|{USA, 40.55149841...|
+--------------------+--------------------+--------------------+



In [ ]:
jdf.createOrReplaceTempView("weather_data")

query = """
select * from weather_data
"""

jdf.createOrReplaceTempView('weather_data')
spark.sql('select * from weather_data').show(truncate=False)

Py4JError: An error occurred while calling o24.sql. Trace:
py4j.Py4JException: Method sql([class java.lang.String, class [Ljava.lang.Object;]) does not exist
	at py4j.reflection.ReflectionEngine.getMethod(ReflectionEngine.java:321)
	at py4j.reflection.ReflectionEngine.getMethod(ReflectionEngine.java:329)
	at py4j.Gateway.invoke(Gateway.java:274)
	at py4j.commands.AbstractCommand.invokeMethod(AbstractCommand.java:132)
	at py4j.commands.CallCommand.execute(CallCommand.java:79)
	at py4j.ClientServerConnection.waitForCommands(ClientServerConnection.java:182)
	at py4j.ClientServerConnection.run(ClientServerConnection.java:106)
	at java.base/java.lang.Thread.run(Thread.java:829)

